In [1]:
#!/usr/bin/env python
# coding: utf-8

# In[13]:
import math
from search import *

# from notebook import psource, heatmap, gaussian_kernel, show_map, final_path_colors, display_visual, plot_NQueens
# import sys
# sys.setrecursionlimit(10000)


# ## Creating a map of USA as given in the problem

# In[14]:


us_map = UndirectedGraph(dict(
    Austin=dict(LosAngeles=1377, Charlotte=1200, Dallas=195,Boston=1963),
    Bakersville=dict(SantaFe=864,LosAngeles=153,SanFrancisco=283),
    Boston=dict(NewYork=225, Chicago=983, SanFrancisco=3095),
    Charlotte=dict(NewYork=634),
    Chicago=dict(Boston=983,Seattle=2064,SantaFe=1272),
    Dallas=dict(SantaFe=640,NewYork=1548),
    LosAngeles=dict(SanFrancisco=383),
    SanFrancisco=dict(Seattle=807),
    SantaFe=dict(Seattle=1463)
))

us_map.locations = dict(
    Austin = 182,
    Charlotte = 929,
    SanFrancisco = 1230,
    LosAngeles = 1100,
    NewYork = 1368,
    Chicago = 800,
    Seattle = 1670,
    SantaFe = 560,
    Bakersville = 1282,
    Boston = 1551,
    Dallas = 0
)


# ## Creating a heuristic function given in the problem

# In[17]:


class USGraph(GraphProblem):
    def h(self,node):
        '''
            Straight line distances between cities is used as the heuristic in this problem
        '''
        locs = getattr(self.graph,'locations',None)
        
        if locs:
            
            if type(node) is str:
                return locs[node]
            return int(locs[node.state])
        else:            
            return math.inf


# In[18]:


USMapProblem = USGraph('Seattle', 'Dallas', us_map)


# # A * search implementation for the US map Problem

# In[19]:


def astar_search_show_frontier(problem, h=None):
    """A* search is best-first graph search with f(n) = g(n)+h(n).
    You need to specify the h function when you call astar_search, or
    else in your Problem subclass."""
    h = memoize(h or problem.h, 'h')
    return best_first_graph_search_show_frontier(problem, lambda n: n.path_cost + h(n))


# In[20]:


def best_first_graph_search_show_frontier(problem, f,showFrontier = True):
    """Search the nodes with the lowest f scores first.
    You specify the function f(node) that you want to minimize; for example,
    if f is a heuristic estimate to the goal, then we have greedy best
    first search; if f is node.depth then we have breadth-first search.
    There is a subtlety: the line "f = memoize(f, 'f')" means that the f
    values will be cached on the nodes as they are computed. So after doing
    a best first search you can examine the f values of the path returned."""
    f = memoize(f, 'f')
    node = Node(problem.initial)
    frontier = PriorityQueue('min', f)
    frontier.append(node)
    explored = set()
    while frontier:

        print("Explored list ==>",explored) 
        print("Frontier list ==> ",frontier.heap)
        print()
        node = frontier.pop()
        print("Current node==> ",node.state)
        print("Evaluation Function f(node) = g(node) + h(node) ==> ",f(node))
               
            
        if problem.goal_test(node.state):
            print("Goal reached. Stopping searching")
            return node

        explored.add(node.state)
        for child in node.expand(problem):
            if child.state not in explored and child not in frontier:
                frontier.append(child)
            elif child in frontier:
                if f(child) < frontier[child]:
                    del frontier[child]
                    frontier.append(child)
        
    return None


# In[21]:


print("\n\nPath in A* search solution\n")
route = astar_search_show_frontier(USMapProblem).solution()
route = ['Seattle'] + route

print("Optimal solution path for A* search")
print(route)


# # RBFS for the US map problem

# In[22]:


def recursive_best_first_search_show_frontier(problem, h=None):
    """[Figure 3.26]"""
    h = memoize(h or problem.h, 'h')
    """Rbf sometimes goes too deep so we will take a step to store all the steps into files 
    we are calling it rbfs_missionay_canibals.txt """
    def RBFS(problem, node, f_limit):
        if problem.goal_test(node.state):
            return node, 0  # (The second value is immaterial)
        successors = node.expand(problem)
        if len(successors) == 0:
            return None, math.inf
        for s in successors:
            s.f = max(s.path_cost + h(s), node.f)
        while True:
            # Order by lowest f value
            successors.sort(key=lambda x: x.f)
            best = successors[0]
            if len(successors) > 1:
                alternative = successors[1].f
            else:
                alternative = math.inf
            print("f_limit = "+str(f_limit))
            print("best = "+str(best.f))
            print("alternative = "+str(alternative))
            print("Current city -  "+str(node.state))
            if best.f > f_limit:
                print("Failure! Backtracking")    
                print("\n")
                return None, best.f
            else:
                print("Next city - "+str(successors[0].state))
                print("\n")
            
            result, best.f = RBFS(problem, best, min(f_limit, alternative))
            if result is not None:
                return result, best.f
            
    node = Node(problem.initial)
    node.f = h(node)
    result, bestf = RBFS(problem, node, math.inf)
    return result


# In[23]:


print("\n\n\nPath in RBFS solution\n")
route = recursive_best_first_search_show_frontier(USMapProblem).solution()
route = ['Seattle']+route
print("Optimal route computed by RBFS ",route)


# # Function to check consistency of the given heuristic function

# In[24]:


def checkConsistency(graph):
    for parent_node in graph.graph_dict:
        parent_heuristic = graph.locations[parent_node]
        for child_node in graph.graph_dict[parent_node]:
            child_heuristic = graph.locations[child_node]
            cost_parent_to_child = graph.graph_dict[parent_node][child_node]
            if parent_heuristic < cost_parent_to_child + child_heuristic:
                print("The given heuristic is consitent at  "+ parent_node + " and its child node "+child_node)
            else:
                print("****Heuristic is inconsistent at "+ parent_node + " and its child node "+child_node + "****")


# In[66]:


checkConsistency(us_map)


# In[ ]:








Path in A* search solution

Explored list ==> set()
Frontier list ==>  [(1670, <Node Seattle>)]

Current node==>  Seattle
Evaluation Function f(node) = g(node) + h(node) ==>  1670
Explored list ==> {'Seattle'}
Frontier list ==>  [(2023, <Node SantaFe>), (2864, <Node Chicago>), (2037, <Node SanFrancisco>)]

Current node==>  SantaFe
Evaluation Function f(node) = g(node) + h(node) ==>  2023
Explored list ==> {'Seattle', 'SantaFe'}
Frontier list ==>  [(2037, <Node SanFrancisco>), (2103, <Node Dallas>), (3609, <Node Bakersville>), (2864, <Node Chicago>)]

Current node==>  SanFrancisco
Evaluation Function f(node) = g(node) + h(node) ==>  2037
Explored list ==> {'Seattle', 'SanFrancisco', 'SantaFe'}
Frontier list ==>  [(2103, <Node Dallas>), (2290, <Node LosAngeles>), (2372, <Node Bakersville>), (5453, <Node Boston>), (2864, <Node Chicago>)]

Current node==>  Dallas
Evaluation Function f(node) = g(node) + h(node) ==>  2103
Goal reached. Stopping searching
Optimal solution path for A* search